In [1]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import glob
import xml.etree.ElementTree as ET


In [ ]:
base_path = "B:\\multispectral-ped-detection\\data\\RAW\\kaist-cvpr15\\"

voc_day_annotation_paths = f'{base_path}annotations-xml-new-sanitized\\day\\**\\*.xml'
voc_night_annotation_paths = f'{base_path}annotations-xml-new-sanitized\\night\\**\\*.xml'

day_image_path = f'{base_path}images\\day\\'
night_image_path = f'{base_path}images\\night\\'

voc_day_annotations = glob.glob(voc_day_annotation_paths, recursive=True)
voc_night_annotations = glob.glob(voc_night_annotation_paths, recursive=True)

In [3]:
mapping = {f"{voc_day_annotations.index(annotation)+1:010d}.jpg":annotation for annotation in voc_day_annotations}

In [4]:
mapping = mapping | {f"n_{voc_night_annotations.index(annotation)+1:010d}.jpg":annotation for annotation in voc_night_annotations}

In [5]:
control_test_images = os.listdir("B:\\multispectral-ped-detection\\data\\DOTA_CONTROL\\images\\test")
blended_test_images = os.listdir("B:\\multispectral-ped-detection\\data\\DOTA_BLENDED\\images\\test_custom\\day") + os.listdir("B:\\multispectral-ped-detection\\data\\DOTA_BLENDED\\images\\test_custom\\night")

In [6]:
control_test_image_annotations = [mapping[image] for image in control_test_images]

In [7]:
blended_test_image_annotations = [mapping[image] for image in blended_test_images]

In [8]:
control_test_image_annotations[0]

'B:\\multispectral-ped-detection\\data\\RAW\\kaist-cvpr15\\annotations-xml-new-sanitized\\day\\set06\\V000\\I00848.xml'

In [9]:
occlusions = {}
reverse_mapping = {value: key for key, value in mapping.items()}

for annotation_path in tqdm(blended_test_image_annotations):
    tree = ET.parse(annotation_path)
    objects = tree.findall("object")
    if len(objects) == 1:
        occlusion_element = objects[0].find("occlusion")
        if occlusion_element != None:
            occlusion = occlusion_element.text
            if not occlusion in occlusions:
                occlusions[occlusion] = {}

            object_name = objects[0].find("name").text
            if not object_name in occlusions[occlusion]:
                occlusions[occlusion][object_name] = []

            occlusions[occlusion][object_name].append(reverse_mapping[annotation_path])

100%|██████████| 7808/7808 [02:10<00:00, 59.98it/s]


In [10]:
test_abc = occlusions["2"]
total_rows =sum([len(rows) for rows in test_abc.values()])
print([(key, (len(items)/total_rows)*100, len(items)) for key, items in test_abc.items()])

[('cyclist', 2.955665024630542, 6), ('person', 89.65517241379311, 182), ('people', 5.911330049261084, 12), ('person?', 1.477832512315271, 3)]


In [11]:
occ_2 = occlusions["2"]["person"]
occ_1 = occlusions["1"]["person"]
occ_0 = occlusions["0"]["person"]

In [17]:
items_to_remove = [occ for occ in occ_1 if occ.startswith("n")][0:33]
occ_1 = [occ for occ in occ_1 if occ not in items_to_remove]


In [18]:
items_to_add = [occ for occ in occ_0 if occ.startswith("n")][0:35]
occ_0 = occ_0[0:200] + items_to_add

In [20]:
import shutil
source_folder = "B:\\multispectral-ped-detection\\data\\DOTA_BLENDED\\images\\test"
destination_folder = "B:\\multispectral-ped-detection\\data\\DOTA_BLENDED\\images\\test_custom"

for occ in tqdm(occ_0):
    shutil.copy(os.path.join(source_folder, occ), os.path.join(destination_folder, "occ_0" ,occ))

for occ in tqdm(occ_1):
    shutil.copy(os.path.join(source_folder, occ), os.path.join(destination_folder, "occ_1" ,occ))

for occ in tqdm(occ_2):
    shutil.copy(os.path.join(source_folder, occ), os.path.join(destination_folder, "occ_2" ,occ))

100%|██████████| 182/182 [00:04<00:00, 39.36it/s]


In [21]:

source_folder = "B:\\multispectral-ped-detection\\data\\DOTA_BLENDED\\images\\test"
destination_folder = "B:\\multispectral-ped-detection\\data\\DOTA_BLENDED\\images\\test_custom"
images = os.listdir(source_folder)
day_images = [image for image in images if not image.startswith("n")]
night_images = [image for image in images if image.startswith("n")]

for image in tqdm(day_images):
    os.rename(os.path.join(source_folder, image), os.path.join(destination_folder, "day" ,image))

for image in tqdm(night_images):
    os.rename(os.path.join(source_folder, image), os.path.join(destination_folder, "night" ,image))

100%|██████████| 2501/2501 [00:19<00:00, 131.42it/s]


In [22]:

source_folder = "B:\\multispectral-ped-detection\\data\\DOTA_CONTROL\\images\\test"
destination_folder = "B:\\multispectral-ped-detection\\data\\DOTA_CONTROL\\images\\test_custom"
images = os.listdir(source_folder)
day_images = [image for image in images if not image.startswith("n")]
night_images = [image for image in images if image.startswith("n")]

for image in tqdm(day_images):
    os.rename(os.path.join(source_folder, image), os.path.join(destination_folder, "day" ,image))

for image in tqdm(night_images):
    os.rename(os.path.join(source_folder, image), os.path.join(destination_folder, "night" ,image))

100%|██████████| 2501/2501 [00:34<00:00, 71.93it/s] 
